<a href="https://colab.research.google.com/github/tonnyhideyori/tensorflow/blob/master/NLP_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%tensorflow_version 2.x
import tensorflow as tf

In [2]:
path_to_file='shakespeare.txt'

In [3]:
text=open(path_to_file,'r').read()

In [4]:
print(text[4500:6000])

converted are
  From his low tract and look another way:
    So thou, thy self out-going in thy noon:
    Unlooked on diest unless thou get a son.


                     8
  Music to hear, why hear'st thou music sadly?
  Sweets with sweets war not, joy delights in joy:
  Why lov'st thou that which thou receiv'st not gladly,
  Or else receiv'st with pleasure thine annoy?
  If the true concord of well-tuned sounds,
  By unions married do offend thine ear,
  They do but sweetly chide thee, who confounds
  In singleness the parts that thou shouldst bear:  
  Mark how one string sweet husband to another,
  Strikes each in each by mutual ordering;
  Resembling sire, and child, and happy mother,
  Who all in one, one pleasing note do sing:
    Whose speechless song being many, seeming one,
    Sings this to thee, 'Thou single wilt prove none'.


                     9
  Is it for fear to wet a widow's eye,
  That thou consum'st thy self in single life?
  Ah, if thou issueless shalt hap to die

In [5]:
vocab=sorted(set(text))

In [6]:
len(vocab)

84

In [7]:
char_to_ind={char:ind for ind,char in enumerate(vocab)}
char_to_ind['H']

33

In [8]:
ind_to_char=np.array(vocab)

In [9]:
encoded_text=np.array([char_to_ind[c] for c in text])

In [10]:
encoded_text[43:78]

array([75, 76, 73, 60, 74,  1, 78, 60,  1, 59, 60, 74, 64, 73, 60,  1, 64,
       69, 58, 73, 60, 56, 74, 60,  8,  0,  1,  1, 45, 63, 56, 75,  1, 75,
       63])

In [11]:
encoded_text.shape

(5445609,)

In [12]:
seq_len=120

In [13]:
total_num_seq = len(text)//(seq_len + 1)

In [14]:
total_num_seq

45005

In [15]:
char_dataset=tf.data.Dataset.from_tensor_slices(encoded_text)

In [16]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [17]:
sequences=char_dataset.batch(seq_len+1,drop_remainder=True)

In [18]:
def create_seq_target(seq):
    input_text=seq[:-1]
    target_text=seq[1:]
    return input_text,target_text

In [19]:
dataset=sequences.map(create_seq_target)
dataset

<MapDataset shapes: ((120,), (120,)), types: (tf.int64, tf.int64)>

In [20]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [21]:
batch_size=128

In [22]:
buffer_size=10000
dataset=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [23]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

In [24]:
vocab_size=len(vocab)

In [25]:
embed_dim=64

In [26]:
rnn_neurons=1026

In [27]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [28]:
def sparse_cat_loss (y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

In [30]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    model=Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile('adam',loss=sparse_cat_loss)
    return model

In [31]:
model=create_model(vocab_size=vocab_size,embed_dim=embed_dim,rnn_neurons=rnn_neurons,batch_size=batch_size)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [33]:
for input_example_batch,target_example_batch in dataset.take(1):
  target_example_batch_predictions=model(input_example_batch)

In [34]:
sampled_indices=tf.random.categorical(target_example_batch_predictions[0],num_samples=1)

In [35]:
sampled_indices=tf.squeeze(sampled_indices,axis=-1).numpy()

In [36]:
sampled_indices

array([ 5, 79, 14, 41, 76, 24, 36, 15, 52, 42,  8, 46,  2, 33,  9, 16, 54,
       53, 53, 53, 77,  0, 39, 75, 69, 83, 23, 32,  0, 66, 80, 46, 82, 43,
       62, 49, 71, 48, 16, 83, 52, 30,  5, 22, 47,  7, 71, 66, 44, 23, 75,
        8, 55, 14, 29,  0, 68, 39, 56, 10, 16, 63, 36, 65, 51, 74, 51, 56,
       22, 28, 53,  4,  4, 68, 16, 11, 53, 33, 35, 41, 18, 15, 35,  1, 30,
       39, 53, 35, 80, 15, 23, 62,  0, 36, 48, 58, 47, 66, 80, 44, 48, 50,
       34, 64, 60, 82, 72, 17, 80, 13, 39, 77, 77, 80, 79,  7, 43, 44, 79,
       14])

In [37]:
ind_to_char[sampled_indices]

array(["'", 'x', '3', 'P', 'u', '>', 'K', '4', '[', 'Q', ',', 'U', '!',
       'H', '-', '5', '_', ']', ']', ']', 'v', '\n', 'N', 't', 'n', '}',
       '<', 'G', '\n', 'k', 'y', 'U', '|', 'R', 'g', 'X', 'p', 'W', '5',
       '}', '[', 'E', "'", ';', 'V', ')', 'p', 'k', 'S', '<', 't', ',',
       '`', '3', 'D', '\n', 'm', 'N', 'a', '.', '5', 'h', 'K', 'j', 'Z',
       's', 'Z', 'a', ';', 'C', ']', '&', '&', 'm', '5', '0', ']', 'H',
       'J', 'P', '7', '4', 'J', ' ', 'E', 'N', ']', 'J', 'y', '4', '<',
       'g', '\n', 'K', 'W', 'c', 'V', 'k', 'y', 'S', 'W', 'Y', 'I', 'i',
       'e', '|', 'q', '6', 'y', '2', 'N', 'v', 'v', 'y', 'x', ')', 'R',
       'S', 'x', '3'], dtype='<U1')

In [38]:
epochs=30


In [39]:
from tensorflow.keras.models import load_model

In [40]:
model=create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)

In [41]:
model.load_weights('shakespeare_gen.h5')
model.build(tf.TensorShape([1,None]))

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [49]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):
  num_generate=gen_size
  input_eval=[char_to_ind[s] for s in start_seed]
  input_eval=tf.expand_dims(input_eval,0)
  text_generated=[]
  temperature=temp 
  model.reset_states()
  for i in range(num_generate):
    predictions=model.predict(input_eval)
    predictions=tf.squeeze(predictions,0)
    predictions=predictions/temperature
    predictated_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
    input_eval=tf.expand_dims([predictated_id],0)
    text_generated.append(ind_to_char[predictated_id])
  return (start_seed+"".join(text_generated))   

In [50]:
print(generate_text(model,"JULIET",gen_size=1000))

JULIETThler

  GLOUCESTER. Why he hath thought out shall become my grief.
  RODERIGO. Good morrow, cousin, Is to call a little prince?
    As I scarce, so thou purpose up tifts,
    But leave us as it will proclaim the empire,
    Receiving that their works or boot of blood;
    My sighs awake, a sparrow. Nay, what noise? When hark!
    But still I turn her fiery me to-day';
    And you are closely in our birth the real
    Of earless bloody huntshan, shall I say
  croory! I will take your name.
  OTHELLO. Well, I will to the Prince and speak with thee for proof, and shame-
    Here.  as water is not st.  
  OLIVIA. What hast thou to me?
  CLOWN. Peace! so he shit in this,
    Led him to palacement. Passio, dreay the
    Peter courses from the hopes of his at his uffect
    Big-sparring in her tongue in him, and him hath rear
    On Fife's need-he in the climate.
  STRATO. Widow?
  PAINTER. Here's one that lov'd me
    Thou com'st to th' Othello.          [Reads]  'Panis, Jack, This th